<a href="https://colab.research.google.com/github/cyss93/CP1_team8/blob/%EC%98%A4%EC%B0%BD%EC%98%81/Brandi_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score
from IPython.display import display
from sklearn.model_selection import train_test_split

from matplotlib import style
import matplotlib.pyplot as plt

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import Ridge


import time
import itertools

In [3]:
df_1 = pd.read_csv('바지.csv')
df_2 = pd.read_csv('상의.csv')
df_3 = pd.read_csv('스커트.csv')
df_4 = pd.read_csv('원피스.csv')

In [32]:
df = pd.concat([df_1, df_2, df_3, df_4])

In [34]:
# '원' 제거
df['할인전 가격'] = df['할인전 가격'].str[:-1]
df['할인후 가격'] = df['할인후 가격'].str[:-1]
df['구매개수'] = df['구매개수'].str[:-5]

# ',' 제거
df['할인전 가격'] = df['할인전 가격'].str.replace(',', '').astype('int64')
df['할인후 가격'] = df['할인후 가격'].str.replace(',', '').astype('int64')
df['구매개수'] = df['구매개수'].str.replace(',', '').astype('int64')

# 숫자형으로 변환
df['할인전 가격'] = pd.to_numeric(df['할인전 가격'])
df['할인후 가격'] = pd.to_numeric(df['할인후 가격'])
df['구매개수'] = pd.to_numeric(df['구매개수'])

In [35]:
# '%' 제거
df['할인율'] = df['할인율'].str[:-1]
# 숫자형으로 변환
df['할인율'] = pd.to_numeric(df['할인율'])

In [46]:
# 원핫인코딩(카테고리)
df = pd.get_dummies(data = df, columns = ['카테고리'], prefix = '카테고리')

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6387 entries, 0 to 909
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   이미지링크     6387 non-null   object
 1   배송명       6387 non-null   object
 2   브랜드       6387 non-null   object
 3   상품명       6387 non-null   object
 4   할인율       6387 non-null   int64 
 5   할인전 가격    6387 non-null   int64 
 6   할인후 가격    6387 non-null   int64 
 7   구매개수      6387 non-null   int64 
 8   카테고리_바지   6387 non-null   uint8 
 9   카테고리_상의   6387 non-null   uint8 
 10  카테고리_스커트  6387 non-null   uint8 
 11  카테고리_원피스  6387 non-null   uint8 
dtypes: int64(4), object(4), uint8(4)
memory usage: 474.0+ KB


In [ ]:
# 상수항 추가. 실행이 될 때마다 추가되기 때문에 독립된 셀에서 위로 빼두었습니다.
df = sm.add_constant(df,has_constant="add")

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [50]:
# 특성 설정
features_after_vif = ['할인율',
                      '할인후 가격',
                      '카테고리_바지',
                      '카테고리_상의',
                      '카테고리_스커트',
                      '카테고리_원피스']


target = '구매개수'
x = df[features_after_vif]
y = df[target]

# 분할
X_train ,X_test , y_train , y_test = train_test_split(x,y,train_size=0.7,test_size=0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#Train the MLR / 회귀모델적합
Model = sm.OLS(y_train, X_train)
FittedModel = Model.fit()

print(FittedModel.summary())

# 에러
y_pred = FittedModel.predict(X_test)

# 평가지표 계산
print('평가지표')
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
display(pd.DataFrame([['MSE', mse],['MAE', mae],['RMSE', rmse],['R2', r2]], columns=['Metric', 'Score']))


# VIF 계산
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif["features"] = X_train.columns
print(vif)

(4470, 6) (1917, 6) (4470,) (1917,)
                            OLS Regression Results                            
Dep. Variable:                   구매개수   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     5.507
Date:                Tue, 05 Jul 2022   Prob (F-statistic):           4.64e-05
Time:                        06:50:36   Log-Likelihood:                -40988.
No. Observations:                4470   AIC:                         8.199e+04
Df Residuals:                    4464   BIC:                         8.203e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
할인율          -23

,Metric,Score
0,MSE,3.730746e+06
1,MAE,7.335985e+02
2,RMSE,1.931514e+03
3,R2,1.257566e-02


   VIF Factor  features
0    1.007346       할인율
1    1.390448    할인후 가격
2    9.613424   카테고리_바지
3    8.936141   카테고리_상의
4    3.158534  카테고리_스커트
5    5.188930  카테고리_원피스


- 할인전 가격이 너무 높게 나와서 제외
- 모델 설명력이 너무 부족하게 나옴. 피처수가 많이 부족해서 이런 현상이 발생하는 것 같아 보임.